# DNC DATA EXPERT

# Dinâmica 01 - New York Flights

## CARREGAMENTO DAS BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget

## CARREGAMENTO DOS DADOS

In [2]:
df = pd.read_csv('D:\\Samuel\\Documentos\\Python\\dncPython\\dinamica\\Datasets\\D01\\nyflights.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Samuel\\Documentos\\Python\\dncPython\\dinamica\\Datasets\\D01\\nyflights.csv'

In [ ]:
# Remover a primeira coluna pois o Python já possui indexação
df = df.drop('Unnamed: 0', 1)

In [ ]:
df.head()

## QUESTÕES A SEREM RESPONDIDAS

01 - Quais são as métricas de centralização, variabilidade e quantis dos vôos que atrasam mais que duas horas para decolar? Responda de modo numérico e gráfico.

02 - Quais destinos geraram vôos com o maior número de atrasos na partida? Responda de modo qantitativo e gráfico. (5 piores | Considerar 5 minutos um atraso aceitável)

03 - Para os 5 piores destinos com maior número de voôes atrasados na partida, calcule as estatísticas descritivas da velocidade média durante o vôo.

04 - Aumentar a velocidade média dos voôs pode ser uma solução para diminuir os atrasos? Responda graficamente.

## QUESTÃO 01

### Filtrar a tabela: atraso > 2hrs

In [ ]:
dfDelay = df[df['dep_delay'] > 120]

### Estatísticas descritivas para os atrados

In [ ]:
delayDesc = dfDelay['dep_delay'].describe()

In [ ]:
# Trasnformar as estatísticas descritivas em data frame
delayDesc = pd.DataFrame(delayDesc)

In [ ]:
# Transpor o dataframe e adicionar nas meétricas faltantes
delayDesc = pd.DataFrame.transpose(delayDesc)

In [ ]:
# Adicionar uma coluna para a amplitude
delayDesc['range'] = delayDesc['max'] - delayDesc['min']

In [ ]:
# Adicionar uma coluna para variância
delayDesc['var'] = pow(delayDesc['std'], 2)

In [ ]:
delayDesc

A média de atrasos para os vôos que atrasam mais que duas horas é de 187 minutos com desvio padrão de 75 minutos. A mediana é de 166 minutos. o maior atraso foi de 1301 minutos (21,86 horas... ouch!).

### Análise gráfica dos atrasos maiores que duas horas.

#### Histograma

In [ ]:
# Cria a figura 'vazia'
fig = plt.figure()

# Plota o histograma
plt.hist(dfDelay['dep_delay'], bins = 20)

# Adiciona título e subtítulo
plt.suptitle('Duração dos atrasos na decolagem', fontsize = 16, fontweight = 'bold', x = 0.465)
plt.title('Valores maiores que duas horas', loc = 'left')

# Adiciona título aos eixos 
plt.xlabel('Minutos')
plt.ylabel('Frequência')

#### Box Plot

In [ ]:
# Plot figura em branco
fig = plt.figure()

# Plota o gráfico de box plot

plt.boxplot(dfDelay['dep_delay'])

plt.ylabel('Minutos')

# Remove o valor do eixo x
plt.xticks([1], [''])

# Remove o tick do eixo x
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)

plt.suptitle('Duração do atraso na decolagem', fontsize = 16, fontweight = 'bold', x = 0.445)
plt.title('Valores maiores que duas horas', loc = 'left')

## QUESTÃO 02

### Filtrar tabela com vôos que atrasaram

In [ ]:
dfDest = df[df['dep_delay'] > 5]

### Contar a quantidade de atrasos nos vôos

In [ ]:
delayCounts = dfDest['dest'].value_counts()

In [ ]:
delayCounts = pd.DataFrame(delayCounts)
delayCounts

In [ ]:
# Transformar o índice das linhas em uma coluna
delayCounts['destino'] = delayCounts.index

# Mudar o nome da coluna de contagem
delayCounts.columns = ['atrasos', 'destino']

# Remover os valores do index
delayCounts.reset_index(drop = True, inplace = True)

# Mudar a sequência das colunas
delayCounts = delayCounts[['destino', 'atrasos']]

delayCounts

Os 5 destinos que geraram mais vôos atrasados foram: Chicago (ORD), Atlanta (ATL), Los Angeles (LAX), Orlando (MCO) e São Francisco (SFO).

### Gráfico de barras 

In [ ]:
fig = plt.figure()

plt.bar(delayCounts['destino'], delayCounts['atrasos'])

plt.ylabel('Atrasos')

# Remove o valor do eixo x
plt.xticks([1], [''])

# Remove o tick do eixo x
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)

plt.suptitle('Atraso na decolagem', fontsize = 18, fontweight = 'bold', x = 0.36)
plt.title('Valores agrupados por diferentes aeroportos', loc = 'left')

## QUESTÃO 03

### Filtrar a tabela somente com os piores destinos

In [ ]:
# Seleciona os piores destinos
worstDest = delayCounts.sort_values(by = 'atrasos', ascending = False)[['destino']][0:4]

In [ ]:
# Filtra a tabela original baseados nos destinos
worstDelay = df[df['dest'].isin(worstDest['destino'])]
worstDelay.reset_index(drop = True, inplace = True)

### Calculara velocidade média de cada vôo

In [ ]:
worstDelay = worstDelay.assign(avgVel = (worstDelay['distance'] / worstDelay['air_time'])*60)

In [ ]:
worstDelay

### Agrupar a tabela com os piores destinos

In [ ]:
grouped = worstDelay.groupby(worstDelay['dest'])

### Criar estatísticas descritivas para velocidade média

In [ ]:
grouped.describe()['avgVel']

### Plotar box-plot para os quatro destinos.

In [ ]:
fig = plt.figure()

sns.boxplot(x = worstDelay['dest'], y = worstDelay['avgVel'])

plt.ylabel('')

plt.xlabel('Destino')

plt.suptitle('Velocidade dos piores destinos', fontsize = 18, fontweight = 'bold', x = 0.465)
plt.title('Milhas por hora', loc = 'left')

## QUESTÃO 04 

### Criar tabela com os 'melhores destinos'

In [ ]:
# Seleciona os melhores destinos
bestDest = delayCounts.sort_values(by = 'atrasos', ascending = True)[['destino']][0:5]

In [ ]:
# Filtra a tabela original baseados nos destinos
bestDelay = df[df['dest'].isin(bestDest['destino'])]
bestDelay.reset_index(drop = True, inplace = True)

In [ ]:
# Criar a coluna com a volocidade média
bestDelay = bestDelay.assign(avgVel = (bestDelay['distance'] / bestDelay['air_time'])*60)

In [ ]:
# Criar uma coluna com a inscrição 'best'
bestDelay = bestDelay.assign(desType = 'best')

### Empilhar planilha com destinos bons e ruins.

In [ ]:
# Adicionar coluna com a inscrição 'worst' nos priores destinos
worstDelay = worstDelay.assign(desType = 'worst')

In [ ]:
# Crioar a mesma coluna com inscrição 'worst' nos destinos 'ruins'
allDelay = worstDelay.assign(desType = 'worst')

In [ ]:
allDelay = worstDelay.append(bestDelay.reset_index(drop = True), ignore_index = True)

### Criar gráfico de box plot para explorar as diferenças

In [ ]:
fig = plt.figure()

sns.boxplot(x = allDelay['desType'], y = allDelay['avgVel'])

plt.ylabel('Velocidade média')

plt.xlabel('Tipo de destino')

plt.suptitle('Comparação entre destinos', fontsize = 18, fontweight = 'bold', x = 0.425)
plt.title('Velocidade média durate o vôo', loc = 'left')

As velocidades médias do voôs que mais atrasam e que menos atrasam tende a ser a mesma. Por isso aumentá-la não parece ser uma boa solução inicialmente.